In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# googleplaystore.csv: https://drive.google.com/file/d/1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ/view?usp=drive_link
# googleplaystore_user_reviews.csv: https://drive.google.com/file/d/1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO/view?usp=drive_link
downloaded = drive.CreateFile({'id':"1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ"})
downloaded.GetContentFile('googleplaystore.csv')

In [4]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context -> variable que vamos a utilizar
sc = spark.sparkContext

Ordenar de forma descendente todas las categorías por cantidad de descargas. (⭐⭐)


In [5]:
def has_valid_installs_format(number_installs):
  try:
    int(number_installs)
    return True
  except ValueError:
    return False


In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Me quedo unicamente con el numero de installs (elimino el + final)

In [7]:
category_rdd = rdd.map(lambda x: (x['Category'], x['Installs'].replace('+', '').replace(',', '')))

Filtro las categorias y me quedo con todos los registros que tengan un valor valido de descargas


In [8]:
clean_category_rdd = category_rdd.filter(lambda x: has_valid_installs_format(x[1])).map(lambda x: (x[0], int(x[1])))
unique_category_rdd = clean_category_rdd.reduceByKey(lambda x, y: x + y)
unique_category_rdd.takeOrdered(unique_category_rdd.count(), lambda x: -x[1])

[('GAME', 35086024415),
 ('COMMUNICATION', 32647276251),
 ('PRODUCTIVITY', 14176091369),
 ('SOCIAL', 14069867902),
 ('TOOLS', 11452771915),
 ('FAMILY', 10258263505),
 ('PHOTOGRAPHY', 10088247655),
 ('NEWS_AND_MAGAZINES', 7496317760),
 ('TRAVEL_AND_LOCAL', 6868887146),
 ('VIDEO_PLAYERS', 6222002720),
 ('SHOPPING', 3247848785),
 ('ENTERTAINMENT', 2869160000),
 ('PERSONALIZATION', 2325494782),
 ('BOOKS_AND_REFERENCE', 1921469576),
 ('SPORTS', 1751174498),
 ('HEALTH_AND_FITNESS', 1583072512),
 ('BUSINESS', 1001914865),
 ('FINANCE', 876648734),
 ('EDUCATION', 871452000),
 ('MAPS_AND_NAVIGATION', 724281890),
 ('LIFESTYLE', 537643539),
 ('WEATHER', 426100520),
 ('FOOD_AND_DRINK', 273898751),
 ('DATING', 264310807),
 ('HOUSE_AND_HOME', 168712461),
 ('ART_AND_DESIGN', 124338100),
 ('LIBRARIES_AND_DEMO', 62995910),
 ('COMICS', 56086150),
 ('MEDICAL', 53257437),
 ('AUTO_AND_VEHICLES', 53130211),
 ('PARENTING', 31521110),
 ('BEAUTY', 27197050),
 ('EVENTS', 15973161)]